In [9]:
def timeout(seconds=2, ermes="Timeout!"):
    from signal import signal, alarm, SIGALRM
    from functools import wraps

    class TimeoutError(Exception):
        pass

    def decorator(func):
        def timeout_handler(signum, frame):
            raise TimeoutError(ermes)

        def wrapper(*args, **kwargs):
            signal(SIGALRM, timeout_handler)
            alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

In [23]:
def test(unique_knapsack):
    assert unique_knapsack(0, []) == 0
    assert unique_knapsack(0, [1]) == 0
    assert unique_knapsack(1, [1]) == 1
    assert unique_knapsack(1, [2]) == 0
    assert unique_knapsack(1, [0,0,0]) == 0
    assert unique_knapsack(2, [1,0,1]) == 2
    assert unique_knapsack(10, [1,4,8]) == 9
    assert unique_knapsack(10**4, list(range(141))) == sum(range(141))
    print("OK")

def main(unique_knapsack):
    W, n = map(int, input().split())
    w = list(map(int, input().split()))
    print(unique_knapsack(W, w))

In [24]:
@timeout()
def unique_knapsack1(full_w, w):
    from functools import lru_cache

    @lru_cache(maxsize=600)
    def knapsack(cur_w, i):
        if cur_w <= 0 or i < 0:
            return 0
        val = knapsack(cur_w, i-1)
        if w[i]:
            prev_w = cur_w - w[i]
            if prev_w == 0:
                val = max(val, w[i])
            elif prev_w > 0:
                val = max(val, w[i] + knapsack(prev_w, i-1))
        #print(val, '<--', cur_w, w[:i+1])
        return val

    return knapsack(full_w, len(w)-1)

In [21]:
@timeout(seconds=3)
def unique_knapsack2(W, w):
    w = [cw for cw in w if 0 < cw <= W]
    n = len(w)+1
    W += 1
    b = [[0]*W for _ in range(n)]
    for i in range(1, n):
        wi = w[i-1]
        for cw in range(1, W):
            bb = b[i-1][cw]
            if cw == wi:
                bb = max(bb, wi)
            if cw > wi:
                bb = max(bb, b[i-1][cw-wi]+wi)
            b[i][cw] = bb
    return b[-1][-1]

#unique_knapsack2(1, [1])
test(unique_knapsack2)
#main(unique_knapsack1)

OK


In [39]:
@timeout(seconds=3)
def unique_knapsack3(vol, weights):
    vol1 = vol + 1
    prev, cur = [0]*vol1, [0]*vol1
    for w in weights:
        prev, cur = cur, prev
        for v in range(vol1):
            cur[v] = prev[v] if w > v else max(prev[v], prev[v-w]+w)
    return cur[-1]

#unique_knapsack2(1, [1])
test(unique_knapsack3)
#main(unique_knapsack1)

OK


In [37]:
vol, n = map(int, input().split())
weights = [*map(int, input().split())]
vol1 = vol + 1
prev, cur = [0]*vol1, [0]*vol1
for w in weights:
    prev, cur = cur, prev
    for v in range(vol1):
        cur[v] = prev[v] if w > v else max(prev[v], prev[v-w]+w)
print(cur[-1])

2 2
1
1
